# Yelp Dataset Challenge [HW03]

La presente tarea se divide en la obtención y generación de datos utilizando herramientas como MapReduce como **mrjob**, para luego ser visualizados mediante **jupyter**.

## 1. Extracción y generación de datos (mrjob)

Para esta tarea se hizo uso de los datos ofrecidos por [**YELP Challenge**](https://www.yelp.com/dataset/challenge), en donde se nos pide trabajar con los datos de los archivos `review.json` y `business.json`, de tal manera de generar un archivo **csv** de datos con la siguiente estructura ``` review_id, category, stars```.

Para realizar esta proceso se desarrollaron dos estapas de MapReduce. La primera, la cual recibe las lineas de los archivos (business y review) cargados en el *stdin* del programa, para luego ser identificarlos y así generar los outputs deseados para realizar los *JOIN's* en la etapa 2.
```python
    def mapper_input(self, _, line):
        if 'name' in line:
            yield line['business_id'], ('business', line['name'], line['business_id'], line['categories'], line['latitude'], line['longitude'], line['state'])
        if 'user_id' in line:
            votes = line['useful'] + line['funny'] + line['cool']

            yield line['business_id'], ('reviews', line['review_id'], line['stars'], line['date'], votes)
```

La segunda estapa consiste en un *reduce* que junta a todos los inputs recibidos por medio de sus valores clave **business_id**, logrando así crear el JOIN deseado de estos guardandolos en un archivo **csv**.

```python
    def reducer_01(self, _, value):
        business = dict()
        reviews = []
        for v in value:
            if v[0] == 'business':
                _, business_name, business_id, categories, latitude, longitude, state = v
                business['categories'] = categories
                business['name'] = business_name
                business['id'] = business_id
                business['categories'] = categories
                business['latitude'] = latitude
                business['longitude'] = longitude
                business['state'] = state
            if v[0] == 'reviews':
                _, review_id, stars, date, votes = v
                reviews.append((review_id, stars, date, votes))


        if any(business) and reviews:
            for review_id, stars, date, votes in reviews:
                for category in business['categories']:
                    writer.writerow((review_id,
                                     category,
                                     business['id'].encode("utf-8"),
                                     business['name'].encode("utf-8"),
                                     business['latitude'],
                                     business['longitude'],
                                     date,
                                     stars,
                                     business['state'].encode("utf-8"),
                                     votes
                                     ))
```

## 2. Visualización de datos (PANDAS + Plot.ly)

En el desarrollo de esta sección se daran a entender dos librerías las cuales permiten manipular y visualizar los datos ya extraídos de **Yelp**. Las librerías trabajadas en esta tarea son:
* **Pandas (v.0.20.3)**
* **Plotly (v.2.2.1)**

La primera (pandas) es una librería para el análisis de datos en Python, la cual provee diversas funciones como:
1. Cargas de datos desde archivos (ej: CSV) de forma simple.
2. Potente estructura de datos para el manejo de estos (llamada DataFrame) donde es posible realizar consultas dentro de ellas o obtener columnas/filas de esta.
3. Graficar datos.
4. Etc.

Mientras que la segunda, la cual es ofrecida como un servicio, permite generar reporterías sofisticadas, con la posibilidad de tener un completo control sobre ellas.

Primero importamos la librería **plotly** para realizar visualizaciónes gráficas sobre los datos

In [1]:
import plotly.plotly as py
import plotly.graph_objs as go

Para poder usar el servicio de **plotly** es necesario tener una cuenta en [Plot.ly](https://plot.ly/), donde podremos obtener una **API Key**, la cual nos dará acceso a generar las visualizaciones deseadas.
Una vez obtenida la **API Key**, se puede ejecutar el siguiente comando en la consola de python de tal manera que sea cargada en las credenciales de la libreria (`~/.plotly/.credentials`).

```python
import plotly 
plotly.tools.set_credentials_file(username='username', api_key='apikey')
```
Reemplazando los valores **username** por el nombre de usuario de Plot.ly y **apikey** por la **API Key** asignada.

Luego importamos la librería **pandas** para importar y manipular los datos desde un archivo **csv**.

In [2]:
import pandas as pd
df = pd.read_csv('../result_01_10000.csv')
df.head(10)

,review_id,category,business_id,business_name,latitude,longitude,date,stars,state,votes
0,2aAE1ZVDqMgFDakrl7iRAw,Arts & Entertainment,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2013-01-08,5,NV,0
1,2aAE1ZVDqMgFDakrl7iRAw,Shopping,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2013-01-08,5,NV,0
2,2aAE1ZVDqMgFDakrl7iRAw,Museums,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2013-01-08,5,NV,0
3,2aAE1ZVDqMgFDakrl7iRAw,Art Galleries,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2013-01-08,5,NV,0
4,4OADaU5e2p63-bSPqtyc7w,Arts & Entertainment,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2013-09-03,3,NV,3
5,4OADaU5e2p63-bSPqtyc7w,Shopping,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2013-09-03,3,NV,3
6,4OADaU5e2p63-bSPqtyc7w,Museums,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2013-09-03,3,NV,3
7,4OADaU5e2p63-bSPqtyc7w,Art Galleries,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2013-09-03,3,NV,3
8,4jAJgSWPilBVjRgDxoN_dw,Arts & Entertainment,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2015-08-17,4,NV,1
9,4jAJgSWPilBVjRgDxoN_dw,Shopping,-MhfebM0QIsKt87iDN-FNw,Bellagio Gallery of Fine Art,36.112896,-115.177637,2015-08-17,4,NV,1


Ejecutando la fila anterior se aprecia la facilidad para importar el archivo, donde este es cargado a un objeto (estructura de datos) **DataFrame**, la cual no permite jugar con los datos y en este caso pedir los primeros 10 con:
```python
df.head(10)
```

Importamos también las librerías y subliberías **ipywidgets** para insertar elementos html como *dropdown* o *select*, entre otros.

In [3]:
import ipywidgets as widgets
from ipywidgets import interact

In [4]:
all_categories = df['category'].tolist()
all_categories = sorted(list(set(all_categories)))

dropdown = widgets.Dropdown(options=all_categories, description='Categoria:')


def on_change(change=''):
    val = df[df.category == change]['stars'].tolist()


    data = [go.Histogram(
        x=val,
    )]

    return py.iplot(data, filename='basic-bar')
     

interact(on_change, change=dropdown)

interactive(children=(Dropdown(description=u'Categoria:', options=('Active Life', 'Amusement Parks', 'Apartments', 'Art Galleries', 'Arts & Entertainment', 'Asian Fusion', 'Automotive', 'Bakeries', 'Barbers', 'Beauty & Spas', 'Breakfast & Brunch', 'Burgers', 'Cafes', 'Cajun/Creole', 'Car Dealers', 'Casinos', 'Child Care & Day Care', 'Chinese', 'Coffee & Tea', 'Contractors', 'Cupcakes', 'Custom Cakes', 'Dentists', 'Desserts', 'Dim Sum', 'Donuts', 'Dry Cleaning & Laundry', 'Education', 'Educational Services', 'Electronics', 'Escape Games', 'Ethnic Food', 'Event Photography', 'Event Planning & Services', 'Eyewear & Opticians', 'Fast Food', 'Fitness & Instruction', 'Food', 'Food Stands', 'Fruits & Veggies', 'General Dentistry', 'Gluten-Free', 'Grocery', 'Health & Medical', 'Health Retreats', 'Home Automation', 'Home Services', 'Home Theatre Installation', 'Hotels', 'Hotels & Travel', 'IT Services & Computer Repair', 'Ice Cream & Frozen Yogurt', 'Indian', 'Italian', 'Language Schools', 'Laser Eye Surgery/Lasik', 'Laundry Services', 'Local Services', 'Massage', 'Massage Therapy', 'Meat Shops', 'Mexican', 'Mobile Phone Repair', 'Mobile Phones', 'Museums', 'Nail Salons', 'Nail Technicians', 'Noodles', 'Optometrists', 'Patisserie/Cake Shop', 'Performing Arts', 'Photographers', 'Photography Stores & Services', 'Pizza', 'Preschools', 'Professional Services', 'Psychics & Astrologers', 'Real Estate', 'Reiki', 'Restaurants', 'Seafood Markets', 'Security Systems', 'Session Photography', 'Shades & Blinds', 'Shoe Repair', 'Shopping', 'Shopping Centers', 'Specialty Food', 'Specialty Schools', 'TV Mounting', 'Vegan', 'Vegetarian', 'Venues & Event Spaces', 'Videographers', 'Wedding Planning', 'Yoga'), value='Active Life'), Output()), _dom_classes=('widget-interact',))

<function __main__.on_change>

La función `on_change` se utiliza para el evento lanzado por el **dropdown** de categorias. De esta manera es posible dar una muestra de que es posible interactuar con los datos, logrando así numerosas visualizaciones de estos.

A continuación se filtrán los datos de **df**, con el fin de crear un set de datos que contenga los negocios, el total de calificaciones por negocio y el promedio de estas para cada uno de ellos.

In [5]:
scatter_data = df[['business_name', 'stars']].groupby('business_name').agg(['count', 'mean']).reset_index()
scatter_data.head(10)

business_name stars          
                                count      mean
0         Advanced Body Therapy    24  5.000000
1               Andy's Cleaners     9  2.333333
2  Bellagio Gallery of Fine Art   320  3.300000
3       Caesars Palace Weddings    26  4.307692
4                Cajun Crawfish    88  2.954545
5     Commercial Shoe & Luggage    10  5.000000
6              Diho Supermarket    35  4.400000
7                Dunkin' Donuts   102  2.352941
8                     Elite A/V   730  4.917808
9                 Escape or Die   178  4.606742

Como se mostró anteriormente, con **pandas** es posible manipular los datos especificando la columna deseada, donde además es posible formular filtros y funciones sobre los datos por medio de metodos *groupby*, *count*, *mean*, entre otros.
```python
scatter_data = df[['business_name', 'stars']].groupby('business_name').agg(['count', 'mean']).reset_index()
```

Finalmente los datos son cargados en un grafico de dispersión (scatter) con el fin de ver a los negocios mejor calificados con respecto a la cantidad de calificaciones obtenidas.

In [6]:

data = []
for idx, sc in scatter_data.iterrows():
    trace = go.Scatter(
    x=sc[('stars', 'count')],
    y=sc[('stars', 'mean')],
    text=sc['business_name'].to_string(),
    name=sc['business_name'].to_string(),
    mode='markers',
    
    )
    data.append(trace)


# Plot and embed in ipython notebook!
py.iplot(data, filename='basic-scatter')

Por ultimo se filtran los datos por el nombre del negocio con el fin de visualizar el promedio de notas obtenido para cada uno en las fechas que fueron calificados.
```python
    val = df[df.business_name == change][['stars','date']].groupby(['date']).mean().reset_index()
```

In [8]:
business_names = df['business_name'].tolist()
business_names = sorted(list(set(business_names)))

dropdown2 = widgets.Dropdown(options=business_names, description='Negocios:')


def on_change2(change=''):
    val = df[df.business_name == change][['stars','date']].groupby(['date']).mean().reset_index()
    stars = val['stars'].values.tolist()
    date = val['date'].values.tolist()
    data = [go.Scatter(
        x = date,
        y = stars,
        fill='tozeroy',
    mode= 'none'
        )]

    
    
    return py.iplot(data, filename='basic-line')
    
    #return lgn.histogram(values=val, bins=5, zoom=False,)

interact(on_change2, change=dropdown2)

interactive(children=(Dropdown(description=u'Negocios:', options=('Advanced Body Therapy', "Andy's Cleaners", 'Bellagio Gallery of Fine Art', 'Caesars Palace Weddings', 'Cajun Crawfish', 'Commercial Shoe & Luggage', 'Diho Supermarket', "Dunkin' Donuts", 'Elite A/V', 'Escape or Die', 'Great Steak & Potato', 'High Roller', 'Hot Yoga Bliss Vacations', 'Jack In The Box', 'Jade At the Palazzo', 'Jeff Mitchum Galleries', 'Las Vegas Chinese School', "Leatherby's Ice Cream Shop", 'Leo Barber Shop', 'Loft 5', 'Mallinger Family Eye Care', 'MetroPCS', 'Mi Pueblo Taco Shop', 'Mindy Bean Photography', 'Miracle Dental', 'Nails By Vicky', 'Pizza Company', 'Settebello Pizzeria Napoletana', 'Sholay', "Sloppi Jo's Roving Eatery", 'Slots A Fun', 'Southern Cove', 'Starbucks', 'Super Summer Theatre', 'Team Ford Lincoln', 'The Shoppes at The Palazzo', 'Veganbites Bakery & Espresso', 'White Castle', 'WuHu Noodle'), value='Advanced Body Therapy'), Output()), _dom_classes=('widget-interact',))

<function __main__.on_change2>

Finalmente se obtiene una noción de las librerias y de sus potenciales, con lo cual nos permiten trabajar sobre datos, manipularlos y visualizarlos en pocos pasos.

### VICENTE NAZAR M.